# Desafio Python

### Importação das bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [ ]:
df = pd.read_csv('googleplaystore.csv')
df = df.drop_duplicates()

df['Installs'] = df['Installs'].astype(str).str.replace(',', '').str.replace('+', '')
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')
df = df.dropna(subset=['Installs'])

In [ ]:
cinco_apps_mais_instalados = df[['App', 'Installs']].sort_values(by='Installs', ascending=False).head(5)

plt.figure(figsize=(10,6))
plt.bar(cinco_apps_mais_instalados['App'], cinco_apps_mais_instalados['Installs'], color='skyblue')

plt.title('5 Aplicativos Mais Instalados')
plt.xlabel('Aplicativo')
plt.ylabel('Número de Instalações')

plt.tight_layout()
plt.show()

In [ ]:
category_counts = df['Category'].value_counts()
plt.figure(figsize=(35, 35))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribuição de Categorias de Aplicativos')
plt.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
# ??????????
df['Price'] = df['Price'].astype(str).str.replace('$', '')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Encontra o app mais caro
app_mais_caro = df[['App', 'Price']].loc[df['Price'].idxmax()]

# Imprime resultados
print(f"Nome: {app_mais_caro['App']}")
print(f"Preço: {app_mais_caro['Price']}$")

In [ ]:
# Garante que a coluna 'Reviews' é numérica
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

# Remove apps duplicados
df_sem_duplicados = df.sort_values(by='Reviews', ascending=False).drop_duplicates(subset='App', keep='first')

# Seleciona os 10 apps com mais reviews
top10_reviews = df_sem_duplicados[['App', 'Reviews']].sort_values(by='Reviews', ascending=False).head(10)

# Imprime resultados
for i, row in top10_reviews.iterrows():
    print(f"Nome: {row['App']} | Número de reviews: {int(row['Reviews']):,} \n")

### Média de preço dos apps pagos

In [ ]:
apps_pagos = df[df['Type'] == 'Paid']
media_preco = apps_pagos['Price'].mean()
print(f"Média de preço dos apps pagos: ${media_preco:.2f}")

In [ ]:
plt.figure(figsize=(10, 1.5))
plt.barh(['Média de Preço'], [media_preco], color='skyblue')
plt.xlim(0, apps_pagos['Price'].max() + 5)
plt.title('Média de Preço dos Apps Pagos')
plt.xlabel('Preço ($)')

# Adiciona o valor da média como texto na barra
plt.text(media_preco + 0.2, 0, f"${media_preco:.2f}", va='center', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.show()

### 5 categorias com maior número de apps

In [ ]:
# Conta número de apps por categoria
apps_por_categoria = df['Category'].value_counts()
cinco_categorias_mais_apps = apps_por_categoria.head(5)

# Imprime resultados
print("5 categorias com mais apps:")
print(cinco_categorias_mais_apps.head(5))

In [ ]:
# Cria gráfico de barras
plt.figure(figsize=(8, 5))
cinco_categorias_mais_apps.plot(kind='bar', color='skyblue')

plt.title('5 Categorias com Mais Aplicativos')
plt.ylabel('Quantidade de Apps')
plt.xlabel('Categoria')
plt.xticks(rotation=0)

# Adiciona valores em cima das barras
for i, valor in enumerate(cinco_categorias_mais_apps.values):
    plt.text(i, valor + 2, str(valor), ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
engine = create_engine('sqlite:///apps_google_play.db')

# Salvar DataFrame no banco (substituir se já existir)
df.to_sql('apps', con=engine, if_exists='replace', index=False)

print("Tabela 'apps' criada com sucesso usando SQLAlchemy.")

### a) nome e a categoria dos 10 apps mais instalados.

In [89]:
query_a = """
SELECT App, Category
FROM apps
ORDER BY Installs DESC
LIMIT 10
"""
result_a = pd.read_sql_query(query_a, con=engine)
print(result_a)

                                        App             Category
0                         Google Play Books  BOOKS_AND_REFERENCE
1  Messenger – Text and Video Chat for Free        COMMUNICATION
2                        WhatsApp Messenger        COMMUNICATION
3              Google Chrome: Fast & Secure        COMMUNICATION
4                                     Gmail        COMMUNICATION
5                                  Hangouts        COMMUNICATION
6  Messenger – Text and Video Chat for Free        COMMUNICATION
7                                  Hangouts        COMMUNICATION
8             Skype - free IM & video calls        COMMUNICATION
9              Google Chrome: Fast & Secure        COMMUNICATION


### b) Nome dos apps classificados como 'Mature 17+

In [ ]:
query_b = """
SELECT COUNT(*)
FROM apps
WHERE "Content Rating" = 'Mature 17+'
"""
resultado_check = pd.read_sql_query(query_b, engine)
print("Total apps 'Mature 17+':", resultado_check.iloc[0, 0])

### c) Média de reviews por categoria

In [94]:
query_c = """
SELECT Category, AVG(Reviews) AS media_reviews
FROM apps
GROUP BY Category
ORDER BY media_reviews DESC
"""
result_c = pd.read_sql_query(query_c, con=engine)
print(result_c)

               Category  media_reviews
0                SOCIAL   1.905632e+06
1         COMMUNICATION   1.642824e+06
2                  GAME   1.262745e+06
3           PHOTOGRAPHY   6.344640e+05
4         VIDEO_PLAYERS   6.307439e+05
5         ENTERTAINMENT   4.285650e+05
6              SHOPPING   4.237998e+05
7                 TOOLS   3.240629e+05
8          PRODUCTIVITY   2.519767e+05
9      TRAVEL_AND_LOCAL   2.344522e+05
10  MAPS_AND_NAVIGATION   2.237902e+05
11               FAMILY   2.042059e+05
12      PERSONALIZATION   1.937968e+05
13               SPORTS   1.861046e+05
14            EDUCATION   1.781962e+05
15              WEATHER   1.781065e+05
16   NEWS_AND_MAGAZINES   1.448707e+05
17   HEALTH_AND_FITNESS   1.008013e+05
18  BOOKS_AND_REFERENCE   9.510099e+04
19       FOOD_AND_DRINK   6.186755e+04
20               COMICS   5.638793e+04
21              FINANCE   4.722192e+04
22       HOUSE_AND_HOME   3.493465e+04
23            LIFESTYLE   3.437151e+04
24             BUSINESS  

### d) Apps com preço maior que 20:

In [88]:
query_d = """
SELECT App, Price
FROM apps
WHERE Price > 20
ORDER BY Price DESC
"""
result_d = pd.read_sql_query(query_d, con=engine)
print(result_d)


                                                  App   Price
0                            I'm Rich - Trump Edition  400.00
1                              most expensive app (H)  399.99
2                                          💎 I'm rich  399.99
3                                           I am rich  399.99
4                                      I am Rich Plus  399.99
5                                   I Am Rich Premium  399.99
6                                          I am Rich!  399.99
7                                  I am rich(premium)  399.99
8                                       I Am Rich Pro  399.99
9                      I am rich (Most expensive app)  399.99
10                                          I am Rich  399.99
11                                 I AM RICH PRO PLUS  399.99
12                  I'm Rich/Eu sou Rico/أنا غني/我很有錢  399.99
13                                        Eu Sou Rico  394.99
14                                          I Am Rich  389.99
15      